## BBC - Project
### Data Exploration

In [6]:
DATA_PATH = './data/'

In [7]:
import numpy as np
from Bio import Geo

def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records
    
records = load_geo(DATA_PATH + 'GSE21510_family.soft')

The following cell should be executed multiple times to run through the data

In [8]:
nr = next(records)
print(nr)
print('\n---------------------- ENTITY_ATTRIBUTES:------------------------------\n')
print(nr.entity_attributes)

GEO Type: DATABASE
GEO Id: GeoMiame
Database_email: geo@ncbi.nlm.nih.gov

Database_institute: NCBI NLM NIH

Database_name: Gene Expression Omnibus (GEO)

Database_web_link: http://www.ncbi.nlm.nih.gov/geo

Column Header Definitions


---------------------- ENTITY_ATTRIBUTES:------------------------------

{'Database_name': 'Gene Expression Omnibus (GEO)', 'Database_institute': 'NCBI NLM NIH', 'Database_web_link': 'http://www.ncbi.nlm.nih.gov/geo', 'Database_email': 'geo@ncbi.nlm.nih.gov'}


In [9]:
# re-loading the data
records = load_geo(DATA_PATH + 'GSE21510_family.soft')
series_sample_id = []
sample_titles = []
genes = []
nb_cols = 0
nb_rows = 0
data = []
for r in records:
    rea = r.entity_attributes
    if 'Series_geo_accession' in rea:
        if rea['Series_geo_accession'] == 'GSE21510':
            series_sample_id = rea['Series_sample_id']
            nb_cols = len(series_sample_id)
    if 'Sample_title' in rea:
        sample_titles.append(rea['Sample_title'])
        if 'sample_table_begin' in rea:
            nb_rows = rea['Sample_data_row_count'] 
            data.append(r.table_rows)
                    

data will become a 2d numpy array of shape (54676, 149) == (NB_GENES+1, NB_SAMPLES+1).  
First row will contain the sample ids, first column will contain the ID_REFs of the genes.

In [10]:
all_data = np.ndarray((int(nb_rows)+1, int(nb_cols)+1), dtype=object)
# labels
all_data[0, 0] = 'ID_REF'
all_data[0, 1:] = np.array(series_sample_id)

for i, d in enumerate(data):
    values = np.array(d[1:])
    if (i == 0):
        all_data[1:, 0] = values[:, 0]
    all_data[1:, i+1] = values[:, 1]
data = np.array(all_data)

MemoryError: 

In [ ]:
data.shape

In [ ]:
print(series_sample_id)
print('')
print(samples_geo_accessions)
print('')
print(sample_titles)
print('')
print(len(data), len(data[0]))
print(data[1][0])

In [ ]:
import re

samples = {}
patient_names = []
for i_t, title in enumerate(sample_titles):
    split = title.split(',')
    if(re.search('cancer', title)):
        samples[series_sample_id[i_t]] = 'cancer'
    else:
        samples[series_sample_id[i_t]] = 'normal'
        
    patient_names.append(split[0])
        
samples

In [ ]:
print(sample_titles[104])